randomly assign video pairs to 21 participants from U01 to U21, each participant will be assigned 105 pairs, and save the result to ```database/baseline/video_assignment.csv```

In [1]:
import random
import pandas as pd
import numpy as np
import itertools

In [2]:
users_baseline = [f"U{i:02d}" for i in range(1, 42, 2)]
users_experiment = [f"U{i:02d}" for i in range(2, 43, 2)]

In [3]:
pair_information_path = "/home/hanfang/repos/trajectory-preference-collection-tool/server/database/data_assignment/pair_information.csv"
pair_information = pd.read_csv(pair_information_path)

# n = len(vids)
# There is an error
# video_pairs = {k: sorted([vids[i], vids[j]]) for i in range(n) for j in range(i+1, n) for k in range(int(n*(n-1)/2))}
# video_pairs = {k: sorted(v) for k,v in enumerate(list(itertools.combinations(sorted(vids), 2)))}
video_pairs = {
    rows["pair_id"]: [rows["video1"], rows["video2"]] for _, rows in pair_information.iterrows()
}

# Create a dictionary to keep track of how many times each video pair has been assigned
pair_counts = {key: 0 for key in video_pairs.keys()}

# Create a list to store the final user-video pairs
final_pairs = []

In [4]:
list(video_pairs.items())

[(0, ['mg_demo_2477', 'mg_demo_3362']),
 (1, ['mg_demo_2477', 'mg_demo_3390']),
 (2, ['mg_demo_2477', 'mg_demo_3706']),
 (3, ['mg_demo_2477', 'mg_demo_3864']),
 (4, ['mg_demo_2477', 'mg_demo_3106']),
 (5, ['mg_demo_2477', 'mh_demo_26']),
 (6, ['mg_demo_2477', 'ph_demo_13']),
 (7, ['mg_demo_2477', 'ph_demo_193']),
 (8, ['mg_demo_2477', 'paired_demo_123']),
 (9, ['mg_demo_2477', 'paired_demo_171']),
 (10, ['mg_demo_2477', 'paired_demo_43']),
 (11, ['mg_demo_2477', 'paired_demo_53']),
 (12, ['mg_demo_2477', 'paired_demo_73']),
 (13, ['mg_demo_2477', 'ph_demo_44']),
 (14, ['mg_demo_2477', 'ph_demo_7']),
 (15, ['mg_demo_2477', 'mh_demo_40']),
 (16, ['mg_demo_2477', 'ph_demo_116']),
 (17, ['mg_demo_2477', 'mh_demo_140']),
 (18, ['mg_demo_2477', 'mh_demo_195']),
 (19, ['mg_demo_2477', 'mh_demo_2']),
 (20, ['mg_demo_2477', 'mh_demo_44']),
 (21, ['mg_demo_2477', 'paired_demo_191']),
 (22, ['mg_demo_2477', 'paired_demo_37']),
 (23, ['mg_demo_2477', 'paired_demo_47']),
 (24, ['mg_demo_2477', 'pai

In [5]:
# Loop over each user and assign them 105 pairs
for i in range(len(users_baseline)):
    num_assigned_pairs = 0
    assigned_pairs = []
    print("Assigning pairs to user", users_baseline[i], users_experiment[i])
    
    # Loop until the user has been assigned 105 pairs
    while num_assigned_pairs < 105:
        random_pairs = list(video_pairs.items())
        random.shuffle(random_pairs)
        # Loop over each video pair and check if it has been assigned less than 5 times
        for key, pair in random_pairs:
            if key not in assigned_pairs and pair_counts[key] <= np.mean(list(pair_counts.values())):
                # Assign the pair to the user
                final_pairs.append((users_baseline[i], users_experiment[i], pair[0], pair[1]))
                assigned_pairs.append(key)
                print("Assigned pair", pair[0], pair[1], "to user", users_baseline[i], users_experiment[i])
                num_assigned_pairs += 1
                
                # Increment the pair count
                pair_counts[key] += 1
                
                # Break out of the loop if the user has been assigned 105 pairs
                if num_assigned_pairs == 105:
                    break
                    
            # Break out of the loop if the user has been assigned 105 pairs
            if num_assigned_pairs == 105:
                break

        # Break out of the loop if the user has been assigned 105 pairs
        if num_assigned_pairs == 105:
            break
                

Assigning pairs to user U01 U02
Assigned pair ph_demo_7 paired_demo_47 to user U01 U02
Assigned pair paired_demo_43 ph_demo_15 to user U01 U02
Assigned pair mg_demo_3362 ph_demo_116 to user U01 U02
Assigned pair ph_demo_44 paired_demo_37 to user U01 U02
Assigned pair mh_demo_140 mh_demo_2 to user U01 U02
Assigned pair mg_demo_3362 ph_demo_7 to user U01 U02
Assigned pair mg_demo_3864 paired_demo_171 to user U01 U02
Assigned pair paired_demo_37 ph_demo_20 to user U01 U02
Assigned pair mg_demo_3362 paired_demo_47 to user U01 U02
Assigned pair mg_demo_2477 mh_demo_140 to user U01 U02
Assigned pair paired_demo_171 mh_demo_140 to user U01 U02
Assigned pair mg_demo_2477 paired_demo_53 to user U01 U02
Assigned pair mg_demo_3390 paired_demo_73 to user U01 U02
Assigned pair paired_demo_73 paired_demo_37 to user U01 U02
Assigned pair mg_demo_3706 paired_demo_123 to user U01 U02
Assigned pair mg_demo_3362 ph_demo_20 to user U01 U02
Assigned pair mg_demo_3706 ph_demo_7 to user U01 U02
Assigned pair

In [6]:
# Create a DataFrame from the final user-video pairs
df = pd.DataFrame(final_pairs, columns=["user_baseline", "user_experiment", "video1", "video2"])

In [7]:
df.to_csv("assigned_user.csv", index=False)

In [8]:
for user_baseline in users_baseline:
    user_df = df[df["user_baseline"] == user_baseline][["video1", "video2"]]
    user_df["pref"] = -1
    user_df.to_csv(f"../baseline/{user_baseline}_pref.csv", index=False)

In [9]:
for user_experiment in users_experiment:
    user_df = pair_information[["video1", "video2"]].copy()
    user_df["pref"] = -1
    user_df.to_csv(f"../experiment/{user_experiment}_pref.csv", index=False)